## 모델 평가와 성능 향상

데이터를 훈련 세트와 테스트 세트로 나누는 이유는 새로운 데이터에 모델이 얼마나 잘 일반화되는지 측정하기 위함이다. 모델이 훈련 세트에 잘 맞는 것보다, 학습 과정에 없던 데이터에 대해 얼마나 잘 예측하는가가 중요하다

## 1. Cross-validation(교차 검증)

- 일반화 성능을 평가하는 데에 트레인/테스트 데이터로 한 번 나누는 것보다 더 안정적이고 뛰어난 통계적 평가 방법
- 교차 검증에서는 데이터를 여러번 반복해서 나누고 여러 모델을 학습함
- 대표적으로 k-fold cross-calidation (k-겹 교차 검증)
- 예를 들어 k=5를 선택한다면,

    1) 데이터를 비슷한 크기의 부분집합 5개로 나눔

    2) 첫번째 모델은 1번 폴드를 테스트 데이터로 사용, 2~5 폴드를 트레인 데이터로 사용 두번째 모델은 2번 폴드를 테스트 데이터로 사용, 1,3,4,5 폴드를 트레인 데이터로 사용

    3) 이와 같이 5개의 정확도 값을 얻게 됨

In [3]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [40]:
iris = datasets.load_iris()

In [41]:
logreg = LogisticRegression(solver = 'lbfgs',multi_class='auto',random_state=0, max_iter=1000)

In [42]:
scores = cross_val_score(logreg, iris.data, iris.target)

print("cross validation scores : {}".format(scores))

cross validation scores : [0.98039216 0.94117647 1.        ]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [43]:
scores_k5 = cross_val_score(logreg, iris.data, iris.target, cv=5)

print("cross validation scores : {}".format(scores_k5))

cross validation scores : [0.96666667 1.         0.93333333 0.96666667 1.        ]


## 교차 검증시 주의 사항
- 단순한 k겹 교차 검증은 데이터가 랜덤하지 않은 경우, 특히 분류 문제에서 클래스 별로 구성된 데이터 일 때, 적용에 주의해야 한다.
- 위와 같은 데이터 형태일 때는 model_selection 모듈 내의 KFold()를 사용하여 기본형 k겹 교차검증을 보완할 수 있다.

In [44]:
from sklearn.model_selection import KFold

In [45]:
Kfold = KFold(n_splits=3)

In [46]:
print("cross validation scores : {}".format(cross_val_score(logreg, iris.data, iris.target, cv=Kfold)))

cross validation scores : [0. 0. 0.]


In [47]:
Kfold1 = KFold(n_splits=3, shuffle=True, random_state=0)

In [48]:
print("cross validation scores : {}".format(cross_val_score(logreg, iris.data, iris.target, cv=Kfold1)))

cross validation scores : [0.98 0.96 0.96]


## 임의분할 계층별 교차검증
- 매우 유연한 교차 검증 전략
- train_size/ test_size로 데이터 임의 분할(중복 없음)
- 이 분할은 n_splits 횟수만큼 반복됨

In [49]:
from sklearn.model_selection import StratifiedShuffleSplit

In [50]:
stratified_shuffle_split = StratifiedShuffleSplit(train_size=0.7,
                                                 test_size=0.3, n_splits = 10,
                                                  random_state= 0)

scores = cross_val_score(logreg, iris.data, iris.target, cv = stratified_shuffle_split)

In [53]:
print("scores : {}".format(scores))

scores : [1.         0.97777778 0.97777778 0.97777778 0.95555556 0.95555556
 0.97777778 0.97777778 0.97777778 0.95555556]


In [55]:
X = iris.data
y = iris.target

for train_index, test_index in stratified_shuffle_split.split(X,y):
    print('Train:',train_index,'\n','Test:',test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

Train: [144 117 115  22  28  38  57 111  91 112  45  56  98  58   1   7 135  69
  80 119 131  96   4  31  93  82   5  63 137 143 140  83 126 132   8  10
  13  73  33  15 130  62 138  16  27  32  52  75  26  95  79  60 108  41
  76  77  87  71 148  11  74 146  84  66 128  94 118  48  59 125 106 124
 145  17 107  40  49 104  97  20  30  25  42  14  92 105 122 116  35 101
   2  18  70  64  72  53  43 114 133  29 127  78  34 120 141] 
 Test: [136 142  39  44  50  23  90 149  46  55  21 147  47 123  67 134  68  65
  51   0  54 103  24  81 129 102 113 100  89 110  99  19   6  88  86 121
  85  37   9  61  36 109   3  12 139]
Train: [102  11 121 119 108 131  52 101  93  91 149  70 126  34 113  66  20  92
  90 141  75  31  81  57  62  99 100 107  74   7 118  67  63  18 146 138
   6  56 148  10  40   5  96   4  14  49  37  83  86  95 115 140 103  59
  41 128  45   8  44  24  71   2  39  55 144 114  64 139  47  13 111  43
  38 134 135 112  51 106 132  33 110 142  69 105  98  23  73  35   3  53
  

In [56]:
print("cross validation scores : {}".format(cross_val_score(logreg, iris.data, iris.target, cv=stratified_shuffle_split)))

cross validation scores : [1.         0.97777778 0.97777778 0.97777778 0.95555556 0.95555556
 0.97777778 0.97777778 0.97777778 0.95555556]


In [58]:
sss = cross_val_score(logreg, iris.data, iris.target, cv=stratified_shuffle_split)

In [59]:
sss.mean()

0.9733333333333334

In [60]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
np.bincount(y_train)

array([35, 35, 35], dtype=int64)

## Grid Search

- 매개변수를 튜닝하여 일반화 성능을 개선하고자 함
- 관심 있는 매개변수들을 대상으로 가능한 조합을 시도하여 최적의 값을 찾음

In [66]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [67]:
iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

print("Train set size:{}\n Test set size:{}".format(X_train.shape[0], X_test.shape[0]))

Train set size:112
 Test set size:38


In [69]:
best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma= gamma, C=C)
        svm.fit(X_train, y_train)
        
        test_score = svm.score(X_test, y_test)
        
        if test_score > best_score:
            best_score = test_score
            best_parameters = {'gamma': gamma, 'C':C}
            train_score = svm.score(X_train, y_train)
            
print('Best Score: {}'.format(np.round(best_score,3)))
print('Train Score: {}'.format(train_score))
print('parameters: {}'.format(best_parameters))

Best Score: 0.974
Train Score: 0.9732142857142857
parameters: {'gamma': 0.001, 'C': 100}


## \<Note\>

- 위의 결과에 따라 모델의 정확도가 97%라고 결론 짓는 것은 위험한 것일수 있음
- 여러가지 매개변수를 조합하여 테스트 세트의 정확도가 가장 높은 조합을 선택했으나,
- 이 정확도는 새로운 데이터에 이어지지 않을 수 있다
- 매개변수를 조정하기 위해 테스트 세트를 이미 사용했기 때문에 최종 모델의 정확성은 새로운 데이터가 생성될 때까지 알 수 없다.
- 즉, 최종 평가를 위해서는 모델을 만들고 튜닝할 때 사용하지 않은 독립된 데이터 셋이 필요하다.
- 애당초 데이터셋을 3개로 나누어 이 문제를 해결 할 수 있다.
    | 훈련세트(train data) | 검증세트(validation data) | 테스트 세트(test data)
- 그러나 이 역시 데이터의 수가 적을때는 부정확 할수 있다.

In [75]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_trainval, X_test, y_trainval, y_test = train_test_split(X,y,test_size=0.25, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(X_trainval,y_trainval,test_size=0.25, random_state=1)

print("Train set size:{}\nValidaion set size:{}\nTest set size:{}".format(X_train.shape[0],X_valid.shape[0], X_test.shape[0]))

Train set size:84
Validaion set size:28
Test set size:38


In [84]:
best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma= gamma, C=C)
        svm.fit(X_train, y_train)
        
        score = svm.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_parameters1 = {'gamma': gamma, 'C':C}
            
svm = SVC(**best_parameters1)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)

print('Best Score in validation: {}'.format(np.round(best_score,3)))
print('parameters1: {}'.format(best_parameters1))
print('Test Score with best parameters: {}'.format(test_score))

Best Score in validation: 0.964
parameters1: {'gamma': 0.001, 'C': 10}
Test Score with best parameters: 0.9210526315789473
